In [ ]:
'''
I/P -> Coder -> <- Peer Reviewer -> Release Manager -> END
Coder <- Release Manager
Peer Reviewer -> Coder
Peer Reviewer -> END
'''

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq


#os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


llm=ChatGroq(model="qwen-2.5-32b")
#llm = ChatOpenAI(model="gpt-4o")
result=llm.invoke("Hello")
result

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 30, 'total_tokens': 40, 'completion_time': 0.05, 'prompt_time': 0.00373977, 'queue_time': 0.237422349, 'total_time': 0.05373977}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_35f92f8282', 'finish_reason': 'stop', 'logprobs': None}, id='run-6db5cf54-a528-4056-8c19-d7a54675086d-0', usage_metadata={'input_tokens': 30, 'output_tokens': 10, 'total_tokens': 40})

In [12]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
# from pydantic import BaseModel


# Graph state
class State(TypedDict):
    topic: str
    code : str
    reviewer : str
    release_manager : str


In [13]:
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(
    content=(
        "You are a Peer Review Agent designed to evaluate Python code for quality and adherence to best practices. Your task is to analyze the provided code snippet and assess its quality based on the following criteria:"

        "1. **Readability**: Is the code easy to read and understand?"
        "2. **Adherence to Standards**: Does the code follow PEP 8 or other relevant coding standards?"
        "3. **Error Handling**: Are there any potential errors or exceptions that are not handled?"
        "4. **Efficiency**: Is the code efficient in terms of performance and resource usage?"
        "5. **Maintainability**: Is the code structured in a way that is easy to maintain and extend?"

        "After reviewing the code, return 'pass' if it meets the quality standards and 'fail' if it does not"

    )
)

#, along with a brief explanation of your assessment."

In [ ]:
# Nodes
def CoderAgent(state: State):
    "Coder class to generate the code"

    msg = llm.invoke(f"Write a code for {state['topic']}")
    return {"code": msg.content}

def PeerReviewer(state: State):
    "PeerReviewer class to review the code"

    msg = llm.invoke(f"Review the cod for {state['topic']}")
    return {"reviewer": msg.content}    

def ReviewerAgent(state: State):

def ReleaseManager(state: State):
    "ReleaseManager class to release the code"

    msg = llm.invoke(f"Release the code for {state['topic']}")
    return {"release_manager": msg.content}


In [18]:
# Build Workflow
workflow = StateGraph(State)

# Add nodes
workflow.add_node("CoderAgent", CoderAgent)
workflow.add_node("PeerReviewer", PeerReviewer)
workflow.add_node("ReleaseManager", ReleaseManager)

# # Add edges to connect nodes
workflow.add_edge(START, "CoderAgent")
workflow.add_conditional_edges("PeerReviewer", {"Fail": "CoderAgent", "Pass": "PeerReviewer"})


# workflow.add_edge("coder", "peer_reviewer")
react_graph=workflow.compile()

# Show
display(Image(react_graph.get_graph().draw_mermaid_png()))

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>